In [3]:
import numpy as np
# from theano.tensor import shared
from theano import shared
import pandas as pd
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.preprocessing import StandardScaler

# the pymc3 version used - 3.11.5
import pymc3 as pm
import matplotlib.pyplot as plt

DATA_DIR = '/final_train_test_split/'
plt.rcParams["font.family"] = "Times New Roman"
plt.style.use('seaborn-ticks')

WARNING (theano.tensor.blas): Using NumPy C-API based implementation for BLAS functions.
/tmp/ipykernel_3267260/2306091604.py:18: MatplotlibDeprecationWarning: The seaborn styles shipped by Matplotlib are deprecated since 3.6, as they no longer correspond to the styles shipped by seaborn. However, they will remain available as 'seaborn-v0_8-<style>'. Alternatively, directly use the seaborn API instead.
  plt.style.use('seaborn-ticks')


In [4]:
def hbm_vary_intercepts_slopes_model(train_df, test_df, selected_names, individual_names, feat_dim):
    """
    This the main function for hierarchical Bayesian model(HBM) used in the paper 
    <PREDICTING BATTERY LIFETIME UNDER VARYING USAGE CONDITIONS FROM EARLY AGING DATA>

    intput_params: 
        train_df - the training set with necessary group/individual features
        test_df - the test set including with necessary group/individual features
        selected_names - all feature column names
        individual_names - individual-level feature column names
        feat_dim - individual-level feature number, will influening the HBM model structure
    outputs:
        tol_rmse_hierarchical - average Root-mean-square-error (RMSE) for predictions on the test set
        tol_mape_hierarchical - average mean-absolute-percentage-error (MAPE) for predictions on the test set

    """

    # Using optimal 4 clusters setting mentioned in the paper
    group_num = len(train_df.cluster_c4.unique())
    with_group_names = selected_names.copy()
    with_group_names.append('cluster_c4')

    x_train, x_test = train_df.loc[:, with_group_names], test_df.loc[:, with_group_names]
    y_train, y_test = train_df.loc[:,'Lifetime'], test_df.loc[:,'Lifetime']

    # calculate group level feature for each cluster
    group_level_variables_train = np.zeros((group_num,1))
    for group_idx in range(group_num):
        mean_gf = np.mean(x_train.loc[x_train.cluster_c4 == group_idx, 'avg_stress'].values)
        group_level_variables_train[group_idx,0] = mean_gf

    raw_group_ind = x_train.loc[:,'cluster_c4'].values
    raw_group_ind = raw_group_ind.astype(int)
    group_ind = shared(np.asarray(raw_group_ind))

    # Standard Scaler for both train and test set
    transformer = StandardScaler()
    processed_x_train = transformer.fit_transform(x_train.loc[:,individual_names])
    processed_x_test = transformer.transform(x_test.loc[:,individual_names])

    # Main HBM construction
    with pm.Model() as hierarchical_model:
        data = pm.Data('data', processed_x_train)
        group_data = pm.Data('group_data', group_level_variables_train)

        g_intercept = pm.Normal('g_intercept', mu = 12, sigma=10)
        g = pm.Normal('g', mu = 0, sigma = 1) # hyperprior intercept
        mu_intercept = g_intercept + g * group_data[:,0]
        intercept = pm.Deterministic('intercept', mu_intercept)

        # hyperprior slope0 - slope2
        g_slope0 = pm.Normal('g_slope0', mu = 0, sigma = 10, shape = 2) 
        mu_slope0 =  g_slope0[0] + g_slope0[1] * group_data[:,0]
        slope_0 = pm.Deterministic('slope_0', mu_slope0)

        g_slope1 = pm.Normal('g_slope1', mu = 0, sigma = 10, shape = 2) 
        mu_slope1 = g_slope1[0] + g_slope1[1] * group_data[:,0] 
        slope_1 = pm.Deterministic('slope_1', mu_slope1)

        # Two model construction choices: 2 or 3 individual-level features
        if feat_dim == 2:
            g_noise = pm.HalfCauchy('g_noise', beta=1)
            noise = pm.HalfCauchy('noise', beta=g_noise, shape=group_num)
            sds = noise[group_ind]
            y_est = intercept[group_ind] + slope_0[group_ind] * data[:,0] + slope_1[group_ind] * data[:,1]
        elif feat_dim == 3:
            g_slope2 = pm.Normal('g_slope2', mu = 0, sigma = 10, shape = 2) # hyperprior slope0 - slope2
            mu_slope2 = g_slope2[0] + g_slope2[1] * group_data[:,0]
            slope_2 = pm.Deterministic('slope_2', mu_slope2)
            g_noise = pm.HalfCauchy('g_noise', beta=1)
            noise = pm.HalfCauchy('noise', beta=g_noise, shape=group_num)
            sds = noise[group_ind]
            y_est = intercept[group_ind] + slope_0[group_ind] * data[:,0] + slope_1[group_ind] * data[:,1] + slope_2[group_ind] * data[:,2]
        else:
            raise ValueError('The individual level features number is either 2 or 3.')
        
        # sampling/inference step, the choice of tune and trace is set to 1000 which is common in analysis
        likelihood = pm.Normal('y_like', mu=y_est, sd=sds, observed=y_train.values)
        hierarchical_trace = pm.sample(1000, tune=1000, target_accept=0.99,return_inferencedata=False)

    with hierarchical_model:
        all_test = processed_x_test
        all_labels = y_test.values

        raw_group_ind = x_test.loc[:,'cluster_c4'].values
        raw_group_ind = raw_group_ind.astype(int)
        group_ind.set_value(np.asarray(raw_group_ind))
        
        pm.set_data({'data': all_test}, model=hierarchical_model)
        all_posterior_samples = pm.sample_posterior_predictive(hierarchical_trace)
        all_pred = all_posterior_samples["y_like"].mean(axis=0)
        tol_mape_hierarchical = mean_absolute_percentage_error(y_true = all_labels, 
                                            y_pred = all_pred)
        tol_rmse_hierarchical = mean_squared_error(y_true= all_labels, 
                                            y_pred = all_pred, squared=False)
                           
    return [tol_rmse_hierarchical, tol_mape_hierarchical, hierarchical_trace, all_pred, all_posterior_samples]

In [5]:
DATA_DIR = 'train_test_split_revised/'
train_df = pd.read_csv(DATA_DIR + 'training.csv',index_col=0)
testin_df = pd.read_csv(DATA_DIR + 'test_in.csv',index_col=0)
testout_df = pd.read_csv(DATA_DIR + 'test_out.csv',index_col=0)

In [7]:
train_df.tail(5)

,Cell,Chg C-rate,Dchg C-rate,DoD,Lifetime,Q_initial,Q_ini_V_low,Q_ini_V_mid,Q_ini_V_high,CV_time_0,...,mean_dqdv_dchg_high_3_0,delta_Q_DVA1,delta_Q_DVA2,delta_Q_DVA3,delta_Q_DVA4,chg_stress,dchg_stress,avg_stress,multi_stress,cluster_c4
Group,,,,,,,,,,,,,,,,,,,,,
G60,G60C4,0.675,1.325,0.828718,9.861,0.284072,-0.078872,-0.116594,-0.089083,1073.0,...,-0.002677,-0.027105,0.014076,0.005979,0.033083,0.747920,1.047880,0.897900,0.783731,1.0
G62,G62C1,1.575,2.425,0.616849,15.050,0.279532,-0.079388,-0.115480,-0.085216,1182.0,...,0.009440,-0.015516,0.016755,0.015432,0.030949,0.985666,1.223053,1.104360,1.205522,0.0
G62,G62C2,1.575,2.425,0.641347,11.719,0.284104,-0.080419,-0.117582,-0.086567,1134.0,...,0.000195,0.004055,0.014574,0.013009,0.008954,1.005048,1.247103,1.126076,1.253399,0.0
G62,G62C3,1.575,2.425,0.632135,11.633,0.279889,-0.079261,-0.116220,-0.085228,1241.0,...,0.005893,-0.020591,0.017067,0.014778,0.035369,0.997804,1.238115,1.117959,1.235396,0.0
G62,G62C4,1.575,2.425,0.677446,7.766,0.284529,-0.074642,-0.116140,-0.094059,948.0,...,0.017968,0.006462,0.018794,0.007393,0.000931,1.032946,1.281720,1.157333,1.323947,0.0


In [5]:
# Calculate the corresponding features mentioned in the paper
# avg_stress: Stress_avg - the group level feature
# log_mean_dqdv_dchg_mid_3_0, log_delta_CV_time_03 are the top two individual-level features mention in Table 1

                                  
train_df['log_mean_dqdv_dchg_mid_3_0'] = np.log(abs(train_df.mean_dqdv_dchg_mid_3_0))
testin_df['log_mean_dqdv_dchg_mid_3_0'] = np.log(abs(testin_df.mean_dqdv_dchg_mid_3_0))
testout_df['log_mean_dqdv_dchg_mid_3_0'] = np.log(abs(testout_df.mean_dqdv_dchg_mid_3_0))

train_df['log_delta_CV_time_03'] = np.log(abs(train_df.delta_CV_time_3_0))
testin_df['log_delta_CV_time_03'] = np.log(abs(testin_df.delta_CV_time_3_0))
testout_df['log_delta_CV_time_03'] = np.log(abs(testout_df.delta_CV_time_3_0))

train_df['log(Lifetime)'] = np.log(train_df.Lifetime)
testin_df['log(Lifetime)'] = np.log(testin_df.Lifetime)
testout_df['log(Lifetime)'] = np.log(testout_df.Lifetime)

In [6]:
# This the experiment on HBM (with 2 individual level features)
# Results recorded in Table 2
selected_names = ['avg_stress', 'log_mean_dqdv_dchg_mid_3_0','log_delta_CV_time_03']
individual_names = [ 'log_mean_dqdv_dchg_mid_3_0','log_delta_CV_time_03']

rmse_hbm_train, mape_hbm_train, sample_trace_train, all_preds_train, posterior_hbm_train = hbm_vary_intercepts_slopes_model(train_df, train_df, selected_names, individual_names, feat_dim=2)
rmse_hbm_testin, mape_hbm_testin, sample_trace_testin, all_preds_testin, posterior_hbm_testin = hbm_vary_intercepts_slopes_model(train_df, testin_df, selected_names, individual_names, feat_dim=2)
rmse_hbm_testout, mape_hbm_testout, sample_trace_testout, all_preds_testout, posterior_hbm_testout  = hbm_vary_intercepts_slopes_model(train_df, testout_df, selected_names, individual_names, feat_dim=2)

print('train set: RMSE-{} MAPE-{}'.format(rmse_hbm_train.round(3), mape_hbm_train.round(3)))
print('testin set: RMSE-{} MAPE-{}'.format(rmse_hbm_testin.round(3), mape_hbm_testin.round(3)))
print('testout set: RMSE-{} MAPE-{}'.format(rmse_hbm_testout.round(3), mape_hbm_testout.round(3)))

# Save the whole posterior for prediction as csv files
# Use them to generate Fig 5 in the paper

np.savetxt('hbm_posterior_train.csv', posterior_hbm_train['y_like'])
np.savetxt('hbm_posterior_testin.csv', posterior_hbm_testin['y_like'])
np.savetxt('hbm_posterior_testout.csv', posterior_hbm_testout['y_like'])

# Save posterior for parameters as csv files
# Use them to generate Fig 5 in the paper

np.savetxt('hbm_intercepts.csv', sample_trace_train['intercept'])
np.savetxt('hbm_slope0s.csv', sample_trace_train['slope_0'])
np.savetxt('hbm_slope1s.csv', sample_trace_train['slope_1'])
np.savetxt('hbm_noises.csv', sample_trace_train['noise'])


Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [noise, g_noise, g_slope1, g_slope0, g, g_intercept]


Sampling 4 chains for 1_000 tune and 1_000 draw iterations (4_000 + 4_000 draws total) took 40 seconds.


Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [noise, g_noise, g_slope1, g_slope0, g, g_intercept]


Sampling 4 chains for 1_000 tune and 1_000 draw iterations (4_000 + 4_000 draws total) took 46 seconds.


Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [noise, g_noise, g_slope1, g_slope0, g, g_intercept]


Sampling 4 chains for 1_000 tune and 1_000 draw iterations (4_000 + 4_000 draws total) took 42 seconds.


train set: RMSE-3.271 MAPE-0.187
testin set: RMSE-3.086 MAPE-0.171
testout set: RMSE-7.318 MAPE-0.217


In [7]:
# This the experiment on HBM (with 3 individual level features)
# Results recorded in Table 2

selected_names = ['avg_stress', 'log_mean_dqdv_dchg_mid_3_0','log_delta_CV_time_03','DoD']
individual_names = [ 'log_mean_dqdv_dchg_mid_3_0','log_delta_CV_time_03','DoD']

rmse_hbm_train, mape_hbm_train, _, _, _ = hbm_vary_intercepts_slopes_model(train_df, train_df, selected_names, individual_names, feat_dim=3)
rmse_hbm_testin, mape_hbm_testin, _, _, _ = hbm_vary_intercepts_slopes_model(train_df, testin_df, selected_names, individual_names, feat_dim=3)
rmse_hbm_testout, mape_hbm_testout, _, _, _ = hbm_vary_intercepts_slopes_model(train_df, testout_df, selected_names, individual_names, feat_dim=3)

print('train set: RMSE-{} MAPE-{}'.format(rmse_hbm_train.round(3), mape_hbm_train.round(3)))
print('testin set: RMSE-{} MAPE-{}'.format(rmse_hbm_testin.round(3), mape_hbm_testin.round(3)))
print('testout set: RMSE-{} MAPE-{}'.format(rmse_hbm_testout.round(3), mape_hbm_testout.round(3)))

Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [noise, g_noise, g_slope2, g_slope1, g_slope0, g, g_intercept]


Sampling 4 chains for 1_000 tune and 1_000 draw iterations (4_000 + 4_000 draws total) took 61 seconds.


Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [noise, g_noise, g_slope2, g_slope1, g_slope0, g, g_intercept]


Sampling 4 chains for 1_000 tune and 1_000 draw iterations (4_000 + 4_000 draws total) took 46 seconds.


Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [noise, g_noise, g_slope2, g_slope1, g_slope0, g, g_intercept]


Sampling 4 chains for 1_000 tune and 1_000 draw iterations (4_000 + 4_000 draws total) took 43 seconds.


train set: RMSE-3.146 MAPE-0.174
testin set: RMSE-2.863 MAPE-0.16
testout set: RMSE-7.51 MAPE-0.241
